In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from IPython.display import display
import skimage.segmentation as seg
import skimage.color as color
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.util import img_as_float

import skimage
from skimage import io
from skimage.color import rgb2gray
import skimage.segmentation as seg
from scipy import ndimage
import skimage.morphology as morph
from skimage.feature import peak_local_max
from skimage import filters
from skimage.morphology import disk, ball
from skimage.util import img_as_ubyte
from skimage import exposure
from skimage.segmentation import flood, flood_fill
from PIL import Image, ImageDraw, ImageFilter
import copy


In [ ]:
grain1_name='Ultra_Co6_2-040.jpg'
grain2_name='Ultra_Co11-2.jpg'
grain3_name='Ultra_Co25-1.jpg'

he_name='he.png'


doggo1_name='doggo1.jpg'
doggo2_name='doggo2.jpg'
doggo3_name='doggo3.jpg'
doggo4_name='doggo4.jpg'
combined_doggo_name='combined_doggo.jpg'

coins_name='coins.jpg'
circ_name='circ.jpg'
grain_zoom_marked_name='grain_zoom_marked.jpg'
grain_zoom_marked_linear_name='grain_zoom_marked_linear.jpg'

path='images'

grain1=io.imread(path+'/'+grain1_name).astype('uint8')
grain2=io.imread(path+'/'+grain2_name).astype('uint8')
grain3=io.imread(path+'/'+grain3_name).astype('uint8')


grain1_pil=Image.open(path+'/'+grain1_name)
grain2_pil=Image.open(path+'/'+grain2_name)
grain3_pil=Image.open(path+'/'+grain3_name)

he=io.imread(path+'/'+he_name).astype('uint8')

coins=io.imread(path+'/'+coins_name).astype('uint8')
circ=io.imread(path+'/'+circ_name).astype('uint8')
grain_zoom_marked=io.imread(path+'/'+grain_zoom_marked_name).astype('uint8')
grain_zoom_marked_linear=io.imread(path+'/'+grain_zoom_marked_linear_name).astype('uint8')


doggo1=io.imread(path+'/'+doggo1_name).astype('uint8')
doggo2=io.imread(path+'/'+doggo2_name).astype('uint8')
doggo3=io.imread(path+'/'+doggo3_name).astype('uint8')
doggo4=io.imread(path+'/'+doggo4_name).astype('uint8')
combined_doggo=io.imread(path+'/'+combined_doggo_name).astype('uint8')

In [ ]:
def img_show(image,N=20):
    
    plt.figure(figsize=(N,N))
    plt.axis('off')
    plt.imshow(image,cmap='gray')
    plt.show()

## Combine images

In [ ]:
def imdivide(image,side):
    
    width,height = image.size
    sides={'left':0,'right':1}
    shape=[(0,0,width//2,height),(width//2,0,width,height)]
    return image.crop(shape[sides[side]])
 

In [ ]:
def combine(image,k=0.5,N=20): 
    left_img=imdivide(image,'left')
    right_img=imdivide(image,'right')

    l=k
    r=1-l
    gray=np.array(left_img)*l
    gray+=np.array(right_img)*r
    gray=gray.astype('uint8')
    return gray

In [ ]:
c2=combine(grain2_pil,0.5,10)
img_show(c2,10)

## Микроструктуры

In [ ]:
img_show(grain1)

In [ ]:
img_show(grain3)

# Otsu's binarization

In [ ]:
def do_otsu(img,N=20):
    image=rgb2gray(img)
    
    global_thresh=skimage.filters.threshold_otsu(image)
    binary_global = image > global_thresh
    

   # ax1.imshow(img,cmap='gray')
    plt.figure(figsize=(N,N))
    plt.imshow(binary_global,cmap='gray')
    plt.show()
    


In [ ]:
#do_otsu(grain1,N=15)
#do_otsu(grain2,N=15)
#do_otsu(grain3,N=15)

do_otsu(grain3,N=15)

In [ ]:
do_otsu(doggo1,N=7)
do_otsu(coins,10)

# Watershed 

Обычный метод

In [ ]:
def do_watershed(image,N=20):
    image=rgb2gray(image)

    segmented = seg.watershed(image)
    fig, ax = plt.subplots(ncols=1, figsize=(N,N), sharex=True, sharey=True)
    
    ax.imshow(segmented, cmap=plt.cm.gray)
    ax.set_title('segmented')

    plt.show()


In [ ]:
do_watershed(grain1,10)
#do_watershed(grain2,10)
#do_watershed(grain3,10)

In [ ]:
do_watershed(coins,7)

Сегментация с маркерами edt (Euclidean Distance Transform)

In [ ]:
def do_watershed_edt(image,N=50):
    image=rgb2gray(image)
    global_thresh=skimage.filters.threshold_otsu(image)
    bin_image = image > global_thresh

    distance = ndi.distance_transform_edt(bin_image)
    distance_copy=copy.copy(distance)
    
    coords = peak_local_max(distance_copy, footprint=np.ones((3, 3)), labels=image.astype('int32')).astype('int64')
    
    mask = np.zeros(distance.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, _ = ndi.label(mask)
    
    labels = seg.watershed(-distance, markers, mask=image)
    labels=labels.astype('uint8')


  
    fig, axes = plt.subplots(ncols=2,nrows=2, figsize=(20,20))
    ax = axes.ravel()
    
    
    ax[0].imshow(image, cmap=plt.cm.gray)
    ax[0].set_title('Overlapping objects')
    
    ax[1].imshow(bin_image, cmap=plt.cm.gray)
    ax[1].set_title('binary image')
    
    ax[2].imshow(distance, cmap=plt.cm.gray)
    ax[2].set_title('Distances')
    
    
    ax[3].imshow(labels, cmap=plt.cm.nipy_spectral)
    ax[3].set_title('Separated objects')
    

#plt.show()

In [ ]:
do_watershed_edt(doggo1,2)

In [ ]:
#do_watershed_edt(grain1,15)

In [ ]:
do_watershed_edt(c2,20)

In [ ]:
do_watershed_edt(grain3,15)

In [ ]:
do_watershed_edt(circ,10)

In [ ]:
do_watershed_edt(coins,10)

Сегментация с гранидентыми маркерами

In [ ]:
def do_watershed_grad(image,N=20):
    image =rgb2gray(image)
    #denoised=image
    denoised = filters.rank.median(image, disk(3))
    #denoised =  filters.rank.equalize(image, disk(50))
    #denoised = exposure.equalize_hist(image/255)

    # find continuous region (low gradient -
    # where less than 10 for this image) --> markers
    # disk(5) is used here to get a more smooth image
    markers = filters.rank.gradient(denoised, disk(8)) < 40
    markers = ndi.label(markers)[0]

    # local gradient (disk(2) is used to keep edges thin)
    gradient = filters.rank.gradient(denoised, disk(1))

    # process the watershed
    labels = seg.watershed(gradient, markers)

    # display results
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 15),
                             sharex=True, sharey=True)
    ax = axes.ravel()

    ax[0].imshow(image, cmap=plt.cm.gray)
    ax[0].set_title("Original")

    ax[1].imshow(gradient, cmap=plt.cm.nipy_spectral)
    ax[1].set_title("Gradient")

    ax[2].imshow(markers, cmap=plt.cm.nipy_spectral)
    ax[2].set_title("Markers")

    ax[3].imshow(image, cmap=plt.cm.gray)
    ax[3].imshow(labels, cmap=plt.cm.nipy_spectral, alpha=.5)
    ax[3].set_title("Segmented")

    for a in ax:
        a.axis('off')

    fig.tight_layout()
    plt.show()

In [ ]:
do_watershed_grad(c2)

In [ ]:
#
# doggo3 - grad 20, disk 8
# doggo2 - grad 25, disk 5
#
do_watershed_grad(doggo3)
#do_watershed_grad(combined_doggo)

In [ ]:
do_watershed_grad(coins)

In [ ]:
do_watershed_grad(grain1)

In [ ]:
do_watershed_grad(grain3)

## Rank filters
https://scikit-image.org/docs/dev/auto_examples/applications/plot_rank_filters.html#sphx-glr-auto-examples-applications-plot-rank-filters-py

In [ ]:
denoised4 = filters.rank.autolevel_percentile(grain1, selem=disk(5), p0=.2, p1=.9)
N=20
plt.figure(figsize=(N,N))
io.imshow(denoised4, cmap=plt.cm.gray)

In [ ]:
denoised4 = filters.rank.enhance_contrast(grain2, disk(2))
N=20
plt.figure(figsize=(N,N))
io.imshow(denoised4, cmap=plt.cm.gray)

In [ ]:
denoised4 = filters.rank.enhance_contrast(image, disk(2))
denoised5 = filters.rank.gradient(grain1, disk(1))
N=20
plt.figure(figsize=(N,N))
io.imshow(denoised5, cmap=plt.cm.gray)

In [ ]:
#denoised5 = filters.rank.entropy(image, disk(2))
denoised5 =  filters.rank.equalize(image, disk(100))
N=20
plt.figure(figsize=(N,N))
io.imshow(denoised5, cmap=plt.cm.gray)

In [ ]:
image =rgb2gray(grain2)
# denoise image
denoised1 = filters.rank.median(image, disk(3))
denoised2 = exposure.equalize_hist(image/255)
denoised3 =  filters.rank.equalize(image, disk(100))
denoised4 = filters.rank.autolevel_percentile(image, selem=disk(20), p0=.2, p1=.7)

N=30

fig, axes = plt.subplots( nrows=5, figsize=(N, N),
                         sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[1].imshow(denoised1, cmap=plt.cm.gray)
ax[2].imshow(denoised2, cmap=plt.cm.gray)
ax[3].imshow(denoised3, cmap=plt.cm.gray)
ax[4].imshow(denoised4, cmap=plt.cm.gray)


plt.show()

# Floodfill


In [ ]:
def do_floodfill(image,N=50):
    
    # Fill a square near the middle with value 127, starting at index (76, 76)
    filled_checkers = flood_fill(image, (76, 76), 127)

    fig, ax = plt.subplots(nrows=2, figsize=(N, N))

    ax[0].imshow(image, cmap=plt.cm.gray)
    ax[0].set_title('Original')

    ax[1].imshow(filled_checkers, cmap=plt.cm.gray)
    ax[1].plot(76, 76, 'wo')  # seed point
    ax[1].set_title('After flood fill')

    plt.show()

In [ ]:
do_floodfill(grain2,N=20)

# K-means

In [ ]:
def do_kmeans(image,N=20,n_segments=100,compactness=10,cmap=plt.cm.gray):
    
    segments = seg.slic(image, n_segments, compactness)
    fig, ax = plt.subplots(nrows=1, figsize=(N, N))



    ax.imshow(segments, cmap=cmap)
    ax.plot(76, 76, 'wo')  # seed point
    ax.set_title('k-means')

    plt.show()

In [ ]:
do_kmeans(grain1,N=15,n_segments=300,compactness=5,cmap=plt.cm.nipy_spectral)

In [ ]:
do_kmeans(grain2,15,500)

# Felzenszwalb

In [ ]:
def do_felzen(image,N=20):
    img = img_as_float(image)
    segments_fz = seg.felzenszwalb(img, scale=100, sigma=0.5, min_size=50)
 
    print(f"Felzenszwalb number of segments: {len(np.unique(segments_fz))}")
    

    fig, ax = plt.subplots(nrows=1, figsize=(N, N), sharex=True, sharey=True)
    felz=seg.mark_boundaries(img, segments_fz)
    ax.imshow(felz)
    ax.set_title("Felzenszwalbs's method")
    plt.tight_layout()
    plt.show()
    return felz 

In [ ]:
felz=do_felzen(grain2,20)

In [ ]:
do_kmeans(felz,15,500)